In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
import hashlib
import time
import math
import random
import pickle
import numpy as np
import tensorflow as tf
import multiprocess as mp
from multiprocess.managers import BaseManager, SyncManager
from libs.TicTacToe import TicTacToe
from libs.model import create_az_model, CachedModel
from libs.MCTS import MCTS
from libs.Trainer import Trainer

In [2]:
__name__

'__main__'

In [3]:
n_cores = mp.cpu_count()
print(f"[Number of cores]: {n_cores}")

[Number of cores]: 16


## Tic-Tac-Toe engine

In [4]:
state = np.array([0., 0., -1., -1., 1., -1., 1., 1., 0.])
TicTacToe().get_available_actions(state)

array([1., 1., 0., 0., 0., 0., 0., 0., 1.])

In [5]:
ttt = TicTacToe()

while ttt.status=="Ongoing":

    valid_moves = ttt.next_moves()
    new_board = random.choice(valid_moves)
    ttt.play(new_board)
    print(np.reshape(ttt.board, (3,3)),"\n")
    if len(valid_moves)==0:
        break

print("[Winner]:",ttt.winner)

[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] 

[[ 0.  1.  0.]
 [ 0.  0.  0.]
 [-1.  0.  0.]] 

[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [-1.  0.  0.]] 

[[ 0.  1. -1.]
 [ 1.  0.  0.]
 [-1.  0.  0.]] 

[[ 1.  1. -1.]
 [ 1.  0.  0.]
 [-1.  0.  0.]] 

[[ 1.  1. -1.]
 [ 1.  0. -1.]
 [-1.  0.  0.]] 

[[ 1.  1. -1.]
 [ 1.  0. -1.]
 [-1.  0.  1.]] 

[[ 1.  1. -1.]
 [ 1.  0. -1.]
 [-1. -1.  1.]] 

[[ 1.  1. -1.]
 [ 1.  1. -1.]
 [-1. -1.  1.]] 

[Winner]: 1


## Create policy and probability model

In [6]:
model = create_az_model()

In [7]:
model.summary()

Model: "tictactoe_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 9)]                  0         []                            
                                                                                                  
 tf.__operators__.add (TFOp  (None, 9)                    0         ['input_1[0][0]']             
 Lambda)                                                                                          
                                                                                                  
 tf.cast (TFOpLambda)        (None, 9)                    0         ['tf.__operators__.add[0][0]']
                                                                                                  
 tf.one_hot (TFOpLambda)     (None, 9, 3)                 0         ['tf.cast[0][0]'

In [8]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


# Monte Carlo Tree Search

In [9]:
model = create_az_model()

In [10]:
model = CachedModel(model=model)

In [11]:
mcts = MCTS(game=TicTacToe(), n_simulations=100)

In [12]:
state = np.array([0., 0., -1., -1., 1., -1., 1., 1., 0.])

In [13]:
root = mcts.run(model=model, state=state, player=1)

1/1 [==============================] - 0s 25ms/step


In [14]:
root

State:
[ 0.  0. -1. -1.  1. -1.  1.  1.  0.]
Player:1
Value:0.9
Leaf:False
Visits:100
Children:
 -0: prior=0.36958505129133085
 -1: prior=0.27528774453589777
 -8: prior=0.3551272041727714

# Generating the first dataset

In [15]:
STARTING_TEMPERATURE = 1

In [16]:
# speed test on 100 games
start_time = time.time()
trainer = Trainer(game = TicTacToe, mcts = MCTS, model=None)
dataset = trainer.create_dataset(number_of_games=100, temperature=STARTING_TEMPERATURE)
print("Running time: %s seconds" % (time.time() - start_time))

Running time: 14.186429500579834 seconds


In [17]:
def parallel_fn(n):
    from libs.Trainer import Trainer
    from libs.TicTacToe import TicTacToe
    from libs.MCTS import MCTS
    N_GAMES_PER_JOB = 25
    STARTING_TEMPERATURE = 1.0
    trainer = Trainer(game = TicTacToe, mcts = MCTS, model=None)
    game_batch = trainer.create_dataset(number_of_games=N_GAMES_PER_JOB, temperature=STARTING_TEMPERATURE)
    return game_batch

In [18]:
start_time = time.time()
ctx = mp.get_context("spawn")
pool = ctx.Pool(n_cores)
dataset = pool.map(parallel_fn, range(n_cores))
print("Running time: %s seconds" % (time.time() - start_time))

Running time: 18.777746438980103 seconds


In [19]:
flatten_dataset = []
for batch in dataset:
    flatten_dataset += batch

In [20]:
flatten_dataset

[(array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0.11, 0.11, 0.11, 0.12, 0.11, 0.11, 0.11, 0.11, 0.11]),
  0),
 (array([-1., -0., -0., -0., -0., -0., -0., -0., -0.]),
  array([0.        , 0.11818182, 0.12727273, 0.11818182, 0.12727273,
         0.12727273, 0.12727273, 0.12727273, 0.12727273]),
  0),
 (array([ 1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([0.        , 0.09734513, 0.        , 0.30973451, 0.09734513,
         0.09734513, 0.09734513, 0.09734513, 0.20353982]),
  0),
 (array([-1., -0.,  1., -0., -0., -1., -0., -0., -0.]),
  array([0.        , 0.14545455, 0.        , 0.13636364, 0.28181818,
         0.        , 0.14545455, 0.14545455, 0.14545455]),
  0),
 (array([ 1.,  0., -1.,  0., -1.,  1.,  0.,  0.,  0.]),
  array([0.        , 0.06153846, 0.        , 0.06153846, 0.        ,
         0.        , 0.75384615, 0.06153846, 0.06153846]),
  0),
 (array([-1., -0.,  1., -0.,  1., -1., -1., -0., -0.]),
  array([0.        , 0.09137056, 0.        , 0.74111675, 0.       

In [21]:
def compute_average_dataset(dataset):
    board_dict = dict()
    policy_dict = dict()
    value_dict = dict()

    for board, policy, value in dataset:
        key = hashlib.sha1(board.view(np.uint8)).hexdigest()

        if key not in board_dict:
            board_dict[key] = board

        if key in policy_dict:
            policy_dict[key].append(policy)
        else:
            policy_dict[key] = [policy]

        if key in value_dict:
            value_dict[key].append(value)
        else:
            value_dict[key] = [value]

    average = []
    for key, board in board_dict.items():
        policies = policy_dict[key]
        values = value_dict[key]
        average.append( (board, sum(policies)/len(policies), sum(values)/len(values)) )

    return average

In [22]:
def windowing_dataset(dataset, length):
    if len(dataset)<=length:
        return dataset
    else:
        return dataset[-length:]

In [23]:
compute_average_dataset(flatten_dataset)

[(array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0.11105 , 0.111175, 0.111425, 0.1111  , 0.1112  , 0.110975,
         0.1114  , 0.1109  , 0.110775]),
  0.3),
 (array([-1., -0., -0., -0., -0., -0., -0., -0., -0.]),
  array([0.        , 0.12493968, 0.12493747, 0.12346327, 0.12493968,
         0.12542887, 0.12592027, 0.12493968, 0.12543108]),
  -0.4594594594594595),
 (array([ 1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([0.        , 0.1039823 , 0.        , 0.17699115, 0.15265487,
         0.1039823 , 0.19247788, 0.1039823 , 0.1659292 ]),
  0.25),
 (array([-1., -0.,  1., -0., -0., -1., -0., -0., -0.]),
  array([0.        , 0.14352373, 0.        , 0.13897828, 0.242679  ,
         0.        , 0.18777152, 0.14352373, 0.14352373]),
  -0.5),
 (array([ 1.,  0., -1.,  0., -1.,  1.,  0.,  0.,  0.]),
  array([0.        , 0.06153846, 0.        , 0.06153846, 0.        ,
         0.        , 0.75384615, 0.06153846, 0.06153846]),
  0.0),
 (array([-1., -0.,  1., -0.,  1., -1., -1., -0., 

# 1st training

In [24]:
trainer = Trainer(game = TicTacToe, mcts = MCTS, model=CachedModel( create_az_model()) )

In [25]:
training_dataset = compute_average_dataset(flatten_dataset)

In [26]:
fit_history = trainer.train(training_dataset, n_epochs=1000, batch_size=30, learning_rate=1e-2)

Epoch 1/1000


38/38 [==============================] - 2s 11ms/step - loss: 3.2098 - policy_loss: 1.7784 - value_loss: 1.3492 - val_loss: 2.7460 - val_policy_loss: 1.9849 - val_value_loss: 0.6610 - lr: 0.0100
Epoch 2/1000
38/38 [==============================] - 0s 5ms/step - loss: 2.1630 - policy_loss: 1.4182 - value_loss: 0.6420 - val_loss: 2.6576 - val_policy_loss: 1.8965 - val_value_loss: 0.6586 - lr: 0.0100
Epoch 3/1000
38/38 [==============================] - 0s 5ms/step - loss: 1.9893 - policy_loss: 1.3013 - value_loss: 0.5874 - val_loss: 2.5240 - val_policy_loss: 1.7611 - val_value_loss: 0.6645 - lr: 0.0100
Epoch 4/1000
38/38 [==============================] - 0s 6ms/step - loss: 1.9119 - policy_loss: 1.2262 - value_loss: 0.5894 - val_loss: 2.6036 - val_policy_loss: 1.8447 - val_value_loss: 0.6646 - lr: 0.0100
Epoch 5/1000
38/38 [==============================] - 0s 5ms/step - loss: 1.8754 - policy_loss: 1.1946 - value_loss: 0.5883 - val_loss: 2.5501 - val_policy_loss: 1.7960 

38/38 [==============================] - 0s 5ms/step - loss: 1.6283 - policy_loss: 0.9945 - value_loss: 0.5871 - val_loss: 1.7374 - val_policy_loss: 1.0298 - val_value_loss: 0.6610 - lr: 2.2823e-04
Epoch 39/1000
38/38 [==============================] - 0s 5ms/step - loss: 1.6281 - policy_loss: 0.9945 - value_loss: 0.5870 - val_loss: 1.7371 - val_policy_loss: 1.0295 - val_value_loss: 0.6611 - lr: 1.7249e-04
Epoch 40/1000
38/38 [==============================] - 0s 5ms/step - loss: 1.6254 - policy_loss: 0.9919 - value_loss: 0.5870 - val_loss: 1.7361 - val_policy_loss: 1.0286 - val_value_loss: 0.6609 - lr: 1.2907e-04
Epoch 41/1000
38/38 [==============================] - 0s 5ms/step - loss: 1.6228 - policy_loss: 0.9893 - value_loss: 0.5870 - val_loss: 1.7354 - val_policy_loss: 1.0280 - val_value_loss: 0.6609 - lr: 1.0000e-04
Epoch 42/1000
38/38 [==============================] - 0s 5ms/step - loss: 1.6253 - policy_loss: 0.9919 - value_loss: 0.5870 - val_loss: 1.7357 - val_policy_loss: 1.0

In [27]:
trainer.model.predict(np.array([[0., 0., -1., -1., 1., -1., 1., 1., 0.]]))

(array([[0.11427562, 0.08511891, 0.09632012, 0.12382594, 0.13353354,
         0.10815345, 0.07497605, 0.15399107, 0.10980526]], dtype=float32),
 array([[0.34428087]], dtype=float32))

In [28]:
trainer.model.predict(np.array([[0., 0., 0., 0., -1., 1., -1., 0., 0.]]))

1/1 [==============================] - 0s 107ms/step


(array([[9.4403282e-02, 9.6316755e-02, 4.3302292e-01, 1.0936800e-01,
         1.6434684e-04, 5.6660181e-04, 1.0533913e-03, 1.1892079e-01,
         1.4618383e-01]], dtype=float32),
 array([[0.13375469]], dtype=float32))

In [29]:
trainer.save_model('saved_model/tmp_model')

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


# Generating the self-playing dataset (2nd) with caching across processes

In [30]:
N_ROUNDS = 40

In [31]:
def parallel_fn(job_n, cache, n):
    
    from libs.Trainer import Trainer
    from libs.TicTacToe import TicTacToe
    from libs.model import CachedModel
    from libs.MCTS import MCTS
    import tensorflow as tf
    N_GAMES_PER_JOB = 25
    STARTING_TEMPERATURE = 10.0
    
    model = tf.keras.models.load_model("saved_model/tmp_model")
    trainer = Trainer(game = TicTacToe, mcts = MCTS, model=CachedModel(model, cache) )
    
    game_batch = trainer.create_dataset(number_of_games=N_GAMES_PER_JOB, temperature=STARTING_TEMPERATURE/n)
    
    return game_batch

In [32]:
if __name__ == '__main__':

    self_play_dataset = flatten_dataset
    
    model = create_az_model()

    for n in range(N_ROUNDS):

        start_time = time.time()
        ctx = mp.get_context("spawn")

        with SyncManager() as manager:

            shared_cache = manager.dict()
            print(f"shared_cache type: {type(shared_cache)}")
            args = [(job_n, shared_cache, n+1) for job_n in range(n_cores)]
            print(f"Number of args: {len(args)}")
            print(args)

            with ctx.Pool(n_cores) as pool:

                round_dataset = pool.starmap(parallel_fn, args)

        print(f"Running time {n+1}-th round: {time.time() - start_time} seconds")

        flatten_round_dataset = []
        for batch in round_dataset:
            flatten_round_dataset += batch
        self_play_dataset += flatten_round_dataset
        training_dataset = compute_average_dataset(self_play_dataset)
        print(f"Dataset size: {len(self_play_dataset)}")

        trainer = Trainer(game = TicTacToe, mcts = MCTS, model=model)
        fit_history = trainer.train(training_dataset, n_epochs=50, batch_size=50, learning_rate=1e-2)

        trainer.model.save("saved_model/tmp_model")
        trainer.model.save("saved_model/model_round_"+str(n))



shared_cache type: <class 'multiprocess.managers.DictProxy'>
Number of args: 16
[(0, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (1, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (2, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (3, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (4, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (5, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (6, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (7, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (8, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (9, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (10, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (11, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (12, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (13, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (14, <DictProxy object, typeid 'dict' at 0x2b8bd8050d0>, 1), (15, <DictProx

38/38 [==============================] - 0s 6ms/step - loss: 0.9895 - policy_loss: 0.8383 - value_loss: 0.0376 - val_loss: 1.3308 - val_policy_loss: 0.6348 - val_value_loss: 0.5826 - lr: 4.9787e-04
Epoch 36/50
38/38 [==============================] - 0s 6ms/step - loss: 0.9825 - policy_loss: 0.8365 - value_loss: 0.0328 - val_loss: 1.3320 - val_policy_loss: 0.6360 - val_value_loss: 0.5830 - lr: 3.8774e-04
INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


INFO:tensorflow:Assets written to: saved_model/model_round_0\assets


INFO:tensorflow:Assets written to: saved_model/model_round_0\assets


...

INFO:tensorflow:Assets written to: saved_model/model_round_38\assets


shared_cache type: <class 'multiprocess.managers.DictProxy'>
Number of args: 16
[(0, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (1, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (2, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (3, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (4, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (5, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (6, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (7, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (8, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (9, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (10, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (11, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (12, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (13, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40), (14, <DictProxy object, typeid 'dict' at 0x2b8e183a880>, 40),

INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


INFO:tensorflow:Assets written to: saved_model/model_round_39\assets


INFO:tensorflow:Assets written to: saved_model/model_round_39\assets
